In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import plotly.offline as ply
import plotly.graph_objs as go

ply.init_notebook_mode(connected=True)

In [4]:
import statsmodels.api as sm

# Считывание данных

Считываем данные, даты, преобразуем в даты, числа в числа

In [5]:
df = pd.read_csv('data.csv', decimal=',')
df['d1'] = pd.to_datetime(df['date_range'].map(lambda s: s.split()[0]), dayfirst=True)
df['d2'] = pd.to_datetime(df['date_range'].map(lambda s: s.split()[2]), dayfirst=True)
df = df[['date_range', 'd1', 'd2', 'avg_price', 'avg_price_conc', 'volume']]
df['volume'] = df['volume'].replace('\xa0', np.NaN).astype('float')

In [6]:
df.head()

,date_range,d1,d2,avg_price,avg_price_conc,volume
0,12.03.2018 - 18.03.2018,2018-03-12,2018-03-18,103197.80,125185.20,197.0
1,19.03.2018 - 25.03.2018,2018-03-19,2018-03-25,107808.30,111489.70,155.0
2,26.03.2018 - 01.04.2018,2018-03-26,2018-04-01,92518.58,98027.54,124.0
3,02.04.2018 - 08.04.2018,2018-04-02,2018-04-08,80068.75,98274.79,131.0
4,09.04.2018 - 15.04.2018,2018-04-09,2018-04-15,97560.45,128452.00,109.0


* date_range - указанный период
* avg_price - Средняя цена «ОАЭ ТУР», руб./заказ
* avg_price_conc - Средняя цена конкурента, руб./заказ
* volume - Факт продаж, заказов

Воспользуемся сервисом yandex wordstat https://wordstat.yandex.ru/ В качестве запроса введём "туры в оаэ". Также можно выбрать варианты группировки по месяцам или по неделям. Хотелось бы выбрать группировку по неделям, но, к сожалению, если так сделать, то он выдает данные только начиная с 09.07.2018 (т.е. примерно за последний год), а если по месяцам, то начиная с 01.07.2017 (т.е. примерно за последние 2 года). Поэтому пришлось группировать данные по месяцам. Весь запрос доступен по ссылке: https://wordstat.yandex.ru/#!/history?regions=213&words=%D1%82%D1%83%D1%80%D1%8B%20%D0%B2%20%D0%BE%D0%B0%D1%8D
Данные были скопированы в файл yandex_data_monthly.csv

#### Считываем данные из yandex wordstat

In [7]:
yandex_df_monthly = pd.read_csv('yandex_data_monthly.csv', decimal=',')

yandex_df_monthly['d'] = \
pd.to_datetime(yandex_df_monthly['period'].map(lambda s: s.split()[0]), dayfirst=True)

yandex_df_monthly['d2'] = \
pd.to_datetime(yandex_df_monthly['period'].map(lambda s: s.split()[2]), dayfirst=True)

yandex_df_monthly = yandex_df_monthly[['period', 'd', 'd2', 'absolute', 'relative']]

In [8]:
yandex_df_monthly.head()

,period,d,d2,absolute,relative
0,01.07.2017 - 31.07.2017,2017-07-01,2017-07-31,23181,0.000004
1,01.08.2017 - 31.08.2017,2017-08-01,2017-08-31,28301,0.000004
2,01.09.2017 - 30.09.2017,2017-09-01,2017-09-30,36927,0.000005
3,01.10.2017 - 31.10.2017,2017-10-01,2017-10-31,49118,0.000006
4,01.11.2017 - 30.11.2017,2017-11-01,2017-11-30,45689,0.000006


* period - промежуток дат
* absolute - абсолютное число запросов
* relative - относительное число запросов

#### Добавляем данные из yandex wordstat

In [9]:
df = df\
.assign(d = lambda df: df['d1'].values.astype('datetime64[M]').astype('datetime64[ns]'))\
.merge(yandex_df_monthly[['d', 'absolute']], how='left', on='d')\
.rename(mapper={'absolute': 'yandex_query'}, axis=1)\
.drop('d', axis=1)

Разумно предположить, что спрос зависит от разности цен. Построим два поля: абсолютная и относительная разницы цен

In [10]:
df['price_dif'] = df['avg_price'] - df['avg_price_conc']
df['price_dif_prc'] = df['price_dif'] / df['avg_price']

Отделяем неразмеченную выборку, продажи преобразуем к int

In [11]:
df_full = df.copy()
df = df[df['volume'].notna()]
df['volume'] = df['volume'].astype('int')

# Первичный анализ

Построим график по изначальным данным: цены "ОАЭ Тур" и конуерентов и объёмы продаж "ОАЭ Тур"

In [12]:
data_plot = [    
    
    go.Scatter(
        x=df_full['d1'],
        y=df_full['avg_price'],
        name='avg_price'
    ),
    
    go.Scatter(
        x=df_full['d1'],
        y=df_full['avg_price_conc'],
        name='avg_price_concurrent'
    ), 
    
    go.Bar(
        x=df_full['d1'],
        y=df_full['volume'],
        name='volume',
        yaxis='y2',
        opacity=0.5
    ),
]

layout = go.Layout(
    yaxis=dict(
        overlaying='y2',        
        title='price'
    ),
    yaxis2=dict(
        side='right',
        title='volume'
    ),    
)

fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

## Однофакторный анализ по каждому признаку 

Построим зависимость спроса от абсолютной разницы цен

In [13]:
data_plot = [    
    
    go.Scatter(
        x=df['d1'],
        y=df['price_dif'],
        name='price_dif'
    ),
    
    
    go.Bar(
        x=df['d1'],
        y=df['volume'],
        name='volume',
        yaxis='y2',
        opacity=0.5
    ),
]

layout = go.Layout(
    yaxis=dict(
        overlaying='y2',        
        title='price'
    ),
    yaxis2=dict(
        side='right',
        title='volume'
    ),    
)

fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

Посмотрим, как однофакторно объёмы продаж зависят от разности цен

In [14]:
X = df[['price_dif']].values
y = df['volume'].values

lr = LinearRegression()
lr.fit(X, y)

vol_pred = lr.predict(X)

data_plot = [    
    
    go.Scatter(
        x=df['price_dif'],
        y=df['volume'],
        mode='markers',
        name='недельное наблюдение'
    ),
    
    go.Scatter(
        x=df['price_dif'],
        y=vol_pred,
        name='regression',
        line=dict(
            dash='dash',
            color='blue'
        )
    ),
]

layout = go.Layout(
    xaxis=dict(
        title='разность цен'
    ),
    yaxis=dict(        
        title='объёмы продаж'
    ),    
)

fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

Построим зависимость спроса от относительной разницы цен

In [15]:
data_plot = [    
    
    go.Scatter(
        x=df['d1'],
        y=df['price_dif_prc'],
        name='price_dif_prc'
    ),
    
    
    go.Bar(
        x=df['d1'],
        y=df['volume'],
        name='volume',
        yaxis='y2',
        opacity=0.5
    ),
]

layout = go.Layout(
    yaxis=dict(
        overlaying='y2',        
        title='price'
    ),
    yaxis2=dict(
        side='right',
        title='volume'
    ),    
)

fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

Посмотрим, как однофакторно объёмы продаж зависят от относительной разности цен

In [16]:
X = df[['price_dif_prc']].values
y = df['volume'].values

lr = LinearRegression()
lr.fit(X, y)

vol_pred = lr.predict(X)

data_plot = [    
    
    go.Scatter(
        x=df['price_dif_prc'],
        y=df['volume'],
        mode='markers',
        name='недельное наблюдение'
    ),
    
    go.Scatter(
        x=df['price_dif_prc'],
        y=vol_pred,
        name='regression',
        line=dict(
            dash='dash',
            color='blue'
        )
    ),
    
]

layout = go.Layout(
    xaxis=dict(
        title='относительная разность цен'
    ),
    yaxis=dict(        
        title='объёмы продаж'
    ),    
)

fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

Построим зависимость спроса от количества запросов в Яндексе

In [17]:
data_plot = [    
     
    go.Scatter(
        x=df['d1'],
        y=df['yandex_query'],
        name='yandex_query'
    ),

    
    go.Bar(
        x=df['d1'],
        y=df['volume'],
        name='volume',
        yaxis='y2',
        opacity=0.5
    ),
]

layout = go.Layout(
    yaxis=dict(
        overlaying='y2',        
        title='yandex_query'
    ),
    yaxis2=dict(
        side='right',
        title='volume'
    ),    
)

fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

Посмотрим, как однофакторно объёмы продаж зависят от относительной разности цен

In [18]:
X = df[['yandex_query']].values
y = df['volume'].values

lr = LinearRegression()
lr.fit(X, y)

vol_pred = lr.predict(X)

data_plot = [    
    
    go.Scatter(
        x=df['yandex_query'],
        y=df['volume'],
        mode='markers',
        name='недельное наблюдение'
    ),
    
    go.Scatter(
        x=df['yandex_query'],
        y=vol_pred,
        name='regression',
        line=dict(
            dash='dash',
            color='blue'
        )
    ),
    
]

layout = go.Layout(
    xaxis=dict(
        title='кол-во запросов в Яндексе'
    ),
    yaxis=dict(        
        title='объёмы продаж'
    ),    
)

fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

## Двухфакторный анализ

Также построим одновременно зависимость и от разницы цен, и от количества запросов в Яндексе

In [19]:
data_plot = [    
    
    go.Scatter(
        x=df['d1'],
        y=df['price_dif'],
        name='price_dif'
    ),
    
    go.Scatter(
        x=df['d1'],
        y=df['yandex_query'],
        name='yandex_query',
        yaxis='y3'
    ),
    
    go.Bar(
        x=df['d1'],
        y=df['volume'],
        name='volume',
        yaxis='y2',
        opacity=0.5
    ),
]

layout = go.Layout(
    yaxis=dict(
        overlaying='y3',        
        title='price'
    ),
    yaxis2=dict(
        side='right',
        title='volume',
        overlaying='y3',        
    ),
    
    yaxis3=dict(
        side='left',
        title='yandex_query'
    ),
)


fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

# Построение модели

Для данной задачи прогнозирования в качестве метода были выбраны линейные регрессии. Это мотивированно тем, что данных очень мало и нужно снизить переобучение к минимуму. По этой же причине признаки для линейных моделей были взяты только самые простые.

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

У нас есть два сильно коррелированных признака: price_dif_prc и price_dif. Чтобы избежать переобучения, выберем из них только один, который имеет большее влияние на модель. Чтобы это сделать, нормируем все признаки, обучим линейную регрессию и оставим тот, у которого вес будет больший по модулю

In [21]:
features = ['price_dif_prc', 'price_dif', 'avg_price', 'yandex_query']

X = scaler.fit_transform(df[features].values)
y = df['volume'].values

lr = LinearRegression()
lr.fit(X, y)

vol_pred = lr.predict(X)

In [22]:
pd.Series(lr.coef_, index=features, name="feature's coefs")

price_dif_prc    -1.159308
price_dif       -19.447179
avg_price         5.589497
yandex_query     38.431285
Name: feature's coefs, dtype: float64

вес у price_dif много больше по модулю, поэтому его и оставим

In [23]:
features = ['price_dif', 'avg_price', 'yandex_query']

X = df[features].values
y = df['volume'].values

lr = LinearRegression()
lr.fit(X, y)

vol_pred = lr.predict(X)

Корень из среднеквадратичной ошибки:

In [24]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df['volume'], vol_pred) ** 0.5

14.160246694580769

Построим график предсказаний модели и имеющихся признаков

In [25]:
data_plot = [    
    
    go.Scatter(
        x=df['d1'],
        y=df['price_dif'],
        name='price_dif'
    ),
    
    go.Scatter(
        x=df['d1'],
        y=df['yandex_query'],
        name='yandex_query',
        yaxis='y3'
    ),
    
    go.Bar(
        x=df['d1'],
        y=df['volume'],
        name='volume',
        yaxis='y2',
        opacity=0.5
    ),
    
    go.Bar(
        x=df['d1'],
        y=vol_pred,
        name='volume_pred',
        yaxis='y2',
        opacity=0.5
    ),    
]

layout = go.Layout(
    yaxis=dict(
        overlaying='y2',        
        title='price'
    ),
    yaxis2=dict(
        side='right',
        title='volume'
    ),    
)


layout = go.Layout(
    yaxis=dict(
        overlaying='y3',        
        title='price'
    ),
    yaxis2=dict(
        side='right',
        title='volume',
        overlaying='y3',        
    ),
    
    yaxis3=dict(
        side='left',
        title='yandex_query'
    ),
)


fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

Как видно, мы уже получили неплохую модель. Но у неё есть недостаток. А именно LinearRegression из пакета sklearn.linear_model предполагает то, что целевая переменная - объём продаж нормально распределена. Но это заведомо неверное предположение, так как объём продаж неотрицателен. Чтобы улучшить модель посмотрим на распределение целевой переменной

In [26]:
data_plot = [
    go.Histogram(
        x=df['volume'],
        nbinsx=20,
        opacity=0.5
    )
]

layout = go.Layout(
    xaxis=dict(
        title='объемы продаж'
    ),
    
    yaxis=dict(        
        title='количество'
    )
)



fig = go.Figure(data=data_plot, layout=layout)
ply.iplot(fig)

При таком распределении естественно считать объём гамма распределённым (а не нормально). Чтобы построить такую линейную модель воспользуемся GLM из пакета statsmodels

Как и ранее проведём отбор из двух признаков price_dif_prc, price_dif.

In [27]:
features = ['price_dif_prc', 'price_dif', 'avg_price', 'yandex_query']

X = df[features].copy()
X.loc[:] = scaler.fit_transform(X.values)

X['const'] = 1
y = df[['volume']]

Фунцкцию связи возьмём логарифмической, таким образом *систематическая компонента модели* $\mu=f(X \beta)$ будет принимать только положительные значения



In [28]:
gamma_model = sm.GLM(y, X, family=sm.families.Gamma(link=sm.families.links.log))

gamma_results = gamma_model.fit()
vol_pred_gamma = gamma_results.mu

In [29]:
gamma_results.params

price_dif_prc   -0.085615
price_dif       -0.080446
avg_price        0.042993
yandex_query     0.352191
const            4.647999
dtype: float64

в данном случае price_dif_prc имеет больший по модулю вес

In [30]:
features = ['price_dif_prc', 'avg_price', 'yandex_query']

X = df[features].copy()
X['const'] = 1
y = df[['volume']]

In [31]:
gamma_model = sm.GLM(y, X, family=sm.families.Gamma(link=sm.families.links.log))

gamma_results = gamma_model.fit()
vol_pred_gamma = gamma_results.mu

In [32]:
print(gamma_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 volume   No. Observations:                   42
Model:                            GLM   Df Residuals:                       38
Model Family:                   Gamma   Df Model:                            3
Link Function:                    log   Scale:                        0.020117
Method:                          IRLS   Log-Likelihood:                -170.48
Date:                Mon, 15 Jul 2019   Deviance:                      0.76463
Time:                        09:50:02   Pearson chi2:                    0.764
No. Iterations:                     9   Covariance Type:             nonrobust
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
price_dif_prc    -0.9896      0.230     -4.298      0.000      -1.441      -0.538
avg_price      1.617e-06   1.88e-06      0.

по коэффициентам модели видно, что основной рыночный фактор это число запросов в Яндексе

Построим график предсказаний обобщенной линейной модели и имеющихся признаков

In [33]:
data_plot = [    
    
    go.Scatter(
        x=df['d1'],
        y=df['price_dif'],
        name='price_dif'
    ),
    
    go.Scatter(
        x=df['d1'],
        y=df['yandex_query'],
        name='yandex_query',
        yaxis='y3'
    ),
    
    go.Bar(
        x=df['d1'],
        y=df['volume'],
        name='volume',
        yaxis='y2',
        opacity=0.5
    ),
    
    go.Bar(
        x=df['d1'],
        y=vol_pred_gamma,
        name='volume_pred_gamma',
        yaxis='y2',
        opacity=0.5
    ),
]



layout = go.Layout(
    yaxis=dict(
        overlaying='y3',        
        title='price'
    ),
    yaxis2=dict(
        side='right',
        title='volume',
        overlaying='y3',        
    ),
    
    yaxis3=dict(
        side='left',
        title='yandex_query'
    ),
)


fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

# Прогноз на неразмеченной части выборки

In [34]:
vol_pred = lr.predict(df_full[features].values)

In [35]:
X = df_full[features].copy()
X['const'] = 1

vol_pred_gamma = gamma_model.predict(gamma_results.params.values, X)

In [36]:
data_plot = [    
    
    go.Scatter(
        x=df_full['d1'],
        y=df_full['price_dif'],
        name='price_dif'
    ),
    
    go.Scatter(
        x=df_full['d1'],
        y=df_full['yandex_query'],
        name='yandex_query',
        yaxis='y3'
    ),
    
    go.Bar(
        x=df_full['d1'],
        y=df_full['volume'],
        name='volume',
        yaxis='y2',
        opacity=0.5
    ),
    
    go.Bar(
        x=df_full['d1'],
        y=vol_pred_gamma,
        name='volume_pred_gamma',
        yaxis='y2',
        opacity=0.5
    ),
    
    go.Bar(
        x=df_full['d1'],
        y=vol_pred,
        name='volume_pred',
        yaxis='y2',
        opacity=0.5
    ),
    
    
]



layout = go.Layout(
    yaxis=dict(
        overlaying='y3',        
        title='price'
    ),
    yaxis2=dict(
        side='right',
        title='volume',
        overlaying='y3',        
    ),
    
    yaxis3=dict(
        side='left',
        title='yandex_query'
    ),
)


fig = go.Figure(data=data_plot, layout=layout)

ply.iplot(fig)

Можно видеть, что обе посроенные регрессии делают примерно одинаковый прогноз. Но в качестве результирующего прогноза будем использовать volume_pred_gamma, поскольку оно,как указано выше лучше описывает данные

Сохраняем прогноз модели в csv файл *data_result.csv*

In [37]:
df_full\
.assign(volume_predict=vol_pred_gamma)\
.to_csv('data_result.csv')